
# Import Python libraries.

In [23]:
# !pip install catboost
# !pip install xgboost==1.3.3

In [24]:
# !pip install imbalanced-learn

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [16]:
import numpy as np # linear algebra
import pandas as pd
from bayes_opt import BayesianOptimization
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor,XGBClassifier
from lightgbm import LGBMRegressor,LGBMClassifier
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestClassifier, BalancedRandomForestClassifier
from sklearn.model_selection import train_test_split,GridSearchCV,StratifiedKFold
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import roc_auc_score,mean_absolute_error,mean_squared_error
from mlxtend.classifier import StackingCVClassifier

from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score,KFold,RepeatedStratifiedKFold,StratifiedKFold

import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN

import os

# Import Datasets

In [17]:
train_data = pd.read_csv(r"train.csv")
test_data = pd.read_csv(r"test.csv")

# Check how train dataset looks like.

In [11]:
train_data.head()

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,single,rented,no,Mechanical_engineer,Rewa,Madhya_Pradesh,3,13,0
1,2,7574516,40,10,single,rented,no,Software_Developer,Parbhani,Maharashtra,9,13,0
2,3,3991815,66,4,married,rented,no,Technical_writer,Alappuzha,Kerala,4,10,0
3,4,6256451,41,2,single,rented,yes,Software_Developer,Bhubaneswar,Odisha,2,12,1
4,5,5768871,47,11,single,rented,no,Civil_servant,Tiruchirappalli[10],Tamil_Nadu,3,14,1


In [108]:
train_data.shape

(252000, 13)

# Convert datatype of selected fields.

In [18]:
train_data["profession"]=pd.factorize(train_data.profession)[0]
train_data["city"]=pd.factorize(train_data.city)[0]
train_data["state"]=pd.factorize(train_data.state)[0]
train_data["married"]=pd.factorize(train_data.married)[0]
train_data["house_ownership"]=pd.factorize(train_data.house_ownership)[0]
train_data["car_ownership"]=pd.factorize(train_data.car_ownership)[0]

In [19]:
test_data["profession"]=pd.factorize(test_data.profession)[0]
test_data["city"]=pd.factorize(test_data.city)[0]
test_data["state"]=pd.factorize(test_data.state)[0]
test_data["married"]=pd.factorize(test_data.married)[0]
test_data["house_ownership"]=pd.factorize(test_data.house_ownership)[0]
test_data["car_ownership"]=pd.factorize(test_data.car_ownership)[0]


In [7]:
train_data

,Id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years,risk_flag
0,1,1303835,23,3,0,0,0,0,0,0,3,13,0
1,2,7574516,40,10,0,0,0,1,1,1,9,13,0
2,3,3991815,66,4,1,0,0,2,2,2,4,10,0
3,4,6256451,41,2,0,0,1,1,3,3,2,12,1
4,5,5768871,47,11,0,0,0,3,4,4,3,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251995,251996,8154883,43,13,0,0,0,46,220,10,6,11,0
251996,251997,2843572,26,10,0,0,0,45,0,0,6,11,0
251997,251998,4522448,46,7,0,0,0,16,106,1,7,12,0
251998,251999,6507128,45,0,0,0,0,22,27,12,0,10,0


In [20]:
test_data

,id,income,age,experience,married,house_ownership,car_ownership,profession,city,state,current_job_years,current_house_years
0,1,7393090,59,19,0,0,0,0,0,0,4,13
1,2,1215004,25,5,0,0,0,1,1,1,5,10
2,3,8901342,50,12,0,0,0,2,2,1,9,14
3,4,1944421,49,9,1,0,1,3,3,1,3,12
4,5,13429,25,18,0,0,1,4,4,0,13,11
...,...,...,...,...,...,...,...,...,...,...,...,...
27995,27996,9955481,57,13,0,0,0,21,78,6,5,10
27996,27997,2917765,47,9,0,0,0,40,149,8,9,14
27997,27998,8082415,24,5,0,0,0,2,179,1,4,13
27998,27999,9474180,51,13,0,0,1,16,12,9,13,14


In [25]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252000 entries, 0 to 251999
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype
---  ------               --------------   -----
 0   Id                   252000 non-null  int64
 1   income               252000 non-null  int64
 2   age                  252000 non-null  int64
 3   experience           252000 non-null  int64
 4   married              252000 non-null  int64
 5   house_ownership      252000 non-null  int64
 6   car_ownership        252000 non-null  int64
 7   profession           252000 non-null  int64
 8   city                 252000 non-null  int64
 9   state                252000 non-null  int64
 10  current_job_years    252000 non-null  int64
 11  current_house_years  252000 non-null  int64
 12  risk_flag            252000 non-null  int64
dtypes: int64(13)
memory usage: 25.0 MB


In [26]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   id                   28000 non-null  int64
 1   income               28000 non-null  int64
 2   age                  28000 non-null  int64
 3   experience           28000 non-null  int64
 4   married              28000 non-null  int64
 5   house_ownership      28000 non-null  int64
 6   car_ownership        28000 non-null  int64
 7   profession           28000 non-null  int64
 8   city                 28000 non-null  int64
 9   state                28000 non-null  int64
 10  current_job_years    28000 non-null  int64
 11  current_house_years  28000 non-null  int64
dtypes: int64(12)
memory usage: 2.6 MB


# Drop the dependent variable from the train/test dataset. 

In [20]:
xtrain=train_data.drop("risk_flag",axis=1)
ytrain=train_data["risk_flag"]

In [21]:
x_train,x_test,y_train,y_test = train_test_split(xtrain,ytrain,test_size=0.2,random_state=101)

# Train your model

In [23]:
# num_folds = 10
# num_instances = len(x_train)
# seed = 7
# scoring = 'roc_auc'

# models = []
# # models.append(('LR', LogisticRegression()))
# # models.append(('LDA', LinearDiscriminantAnalysis()))
# # models.append(('KNN', KNeighborsClassifier()))
# # models.append(('DT', DecisionTreeClassifier()))
# # models.append(('NN', MLPClassifier(solver='lbfgs')))
# # models.append(('NB', GaussianNB()))
# # models.append(('GB', GradientBoostingClassifier()))
# # models.append(('XGB', XGBClassifier()))
# # models.append(('LGB', LGBMClassifier()))
# # models.append(('cat', CatBoostClassifier()))
# models.append(('rr', RandomForestClassifier(max_depth= 42,min_samples_leaf= 2,min_samples_split= 9,n_estimators= 1180)))



# results = []
# names = []
# print("Scores for each algorithm:")
# for name, model in models:
#     resample = SMOTEENN()
#     steps = [('r', resample), ('m', model)]
#     pipeline = Pipeline(steps=steps)
#     kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=41)
#     cv_results = cross_val_score(pipeline, x_train, y_train, cv = kfold, scoring = scoring)
#     results.append(cv_results)
#     names.append(name)
#     print(name, cv_results)
#     print()

## HyperParameter Tuning


# XGBOOST

In [7]:
# space = {
#         'n_estimators':hp.choice('n_estimators', np.arange(400, 1000, 10, dtype=int)),

    
#         'gamma': hp.quniform ('gamma', 0.1,3,0.01),

#         'subsample':hp.quniform('subsample', 0.5, 0.9, 0.01),

#         'eta':hp.quniform('eta', 0.05, 0.5, 0.01)
# }

In [16]:



# def score(params):
    
    
#     resample = SMOTEENN()
#     model = XGBClassifier(**params)
#     steps = [('r', resample), ('m', model)]
#     pipeline = Pipeline(steps=steps)
#     cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=41)

#     cv_results = cross_val_score(pipeline, x_train, y_train, cv = cv, scoring = 'roc_auc')
    
#     score=1-max(cv_results)
    
#     print(score)

#     return {'loss': score, 'status': STATUS_OK}    

# def optimize(trials, space):

#     best = fmin(score, space, algo=tpe.suggest, max_evals=100)

#     return best


# trials = Trials()

# best_params = optimize(trials, space)


# # Return the best parameters

# space_eval(space, best_params)

# xgboost
{'eta': 0.33,
 'gamma': 1.0947024278676265,
 'n_estimators': 690,
 'subsample': 0.77}
 
 {eta': 0.39, 'gamma': 0.77, 'n_estimators': 880, 'subsample': 0.87}
 
 {'eta': 0.2,
 'eval_metric': 'auc',
 'gamma': 0.9,
 'n_estimators': 610,
 'scale_pos_weight': 11,
 'subsample': 0.9500000000000001}
 
 
 {'eta': 0.5,
 'eval_metric': 'auc',
 'gamma': 0.9,
 'n_estimators': 630,
 'scale_pos_weight': 87,
 'subsample': 1.0}
 
 {'eta': 0.375,
 'eval_metric': 'auc',
 'gamma': 0.5,
 'n_estimators': 900,
 'scale_pos_weight': 28,
 'subsample': 0.9}

# LIGHTGBM

In [40]:
# space = {


#         'n_estimators':hp.choice('n_estimators', np.arange(400, 1200, 10, dtype=int)),

    
#         'min_child_weight': hp.uniform ('min_child_weight', 1,30),
    
#         'max_depth':hp.choice('max_depth', np.arange(5, 13, 1, dtype=int)),

#         'subsample':hp.quniform('subsample', 0.3, 0.9, 0.01),

#         'learning_rate':hp.quniform('learning_rate', 0.05, 0.5, 0.01),
        
#         'bagging_fraction': hp.uniform('bagging_fraction',0.5, 1)

        

#     }

In [17]:
# def score(params):

#     model = LGBMClassifier(**params)

#     model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)],

#               verbose=False, early_stopping_rounds=10)

#     Y_pred = model.predict(x_test)

#     score = roc_auc_score(y_test, Y_pred)
# #     score=np.sqrt(-cross_val_score(model,train_X,train_y,cv=kfolds,scoring='neg_mean_squared_error'))

#     print(score)

#     return {'loss': -score, 'status': STATUS_OK}    

# def optimize(trials, space):

#     best = fmin(score, space, algo=tpe.suggest, max_evals=100)

#     return best


# trials = Trials()

# best_params = optimize(trials, space)


# # Return the best parameters

# space_eval(space, best_params)

# lgb parameters
{'bagging_fraction': 0.6985160240622916,
 'learning_rate': 0.46,
 'max_depth': 8,
 'min_child_weight': 5.713456042367998,
 'n_estimators': 700,
 'subsample': 0.71}
 
 {'bagging_fraction': 0.6362408347656274,
 'learning_rate': 0.41000000000000003,
 'max_depth': 12,
 'min_child_weight': 1.9928502540411177,
 'n_estimators': 910,
 'num_leaves': 31,
 'subsample': 0.56}

# CatBoost

In [13]:


# space = {'iterations': hp.choice('iterations',np.arange(10, 1000,1, dtype=int)),
#                  'depth': hp.choice('depth',np.arange(1, 8,1, dtype=int)),
#                  'learning_rate': hp.quniform('learning_rate', 0.05, 0.9, 0.01),
#                  'random_strength': hp.quniform('random_strength',0.01, 10,0.01),
#                  'bagging_temperature': hp.uniform('bagging_temperature',0.0, 1) ,
#                  'border_count': hp.choice('border_count',np.arange(1, 255,1, dtype=int)),
#                  'l2_leaf_reg': hp.choice('l2_leaf_req',np.arange(2, 30,1, dtype=int)),
#                  'class_weights': [0.13, 0.87],
#                  'eval_metric':'AUC'
#         }

In [15]:


# def score(params):

#     model = CatBoostClassifier(**params)

#     model.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test, y_test)],

#               verbose=False, early_stopping_rounds=10)

#     Y_pred = model.predict(x_test)

#     score = roc_auc_score(y_test, Y_pred)
# #     score=np.sqrt(-cross_val_score(model,train_X,train_y,cv=kfolds,scoring='neg_mean_squared_error'))

#     print(score)
    
#     loss=1-score

#     return {'loss': loss, 'status': STATUS_OK}



# def optimize(trials, space):

#     best = fmin(score, space, algo=tpe.suggest, max_evals=100)

#     return best


# trials = Trials()

# best_params = optimize(trials, space)


# # Return the best parameters

# space_eval(space, best_params)

# catboost
{'bagging_temperature': 0.7159912273814812,
 'border_count': 247,
 'depth': 7,
 'iterations': 665,
 'l2_leaf_reg': 20,
 'learning_rate': 0.67,
 'random_strength': 0.11706929793799148,
 'scale_pos_weight': 0.9139659820768006}
 
 {'bagging_temperature': 0.013585295009384768,
 'border_count': 230,
 'depth': 7,
 'iterations': 974,
 'l2_leaf_reg': 18,
 'learning_rate': 0.58,
 'random_strength': 0.65,
 'scale_pos_weight': 0.6291109899364863}
 
 {'bagging_temperature': 0.7212640190796531,
 'border_count': 235,
 'class_weights': (0.13, 0.87),
 'depth': 7,
 'eval_metric': 'AUC',
 'iterations': 871,
 'l2_leaf_reg': 13,
 'learning_rate': 0.17,
 'random_strength': 1.57}

# randomforest

In [62]:
# space= {'max_depth': hp.choice('max_depth', np.arange(10, 100, 1, dtype=int)),
#  'min_samples_leaf':hp.choice('min_samples_leaf', np.arange(1, 10, 1, dtype=int)),
#  'min_samples_split':hp.choice('min_samples_split', np.arange(2, 10, 1, dtype=int)),
#  'n_estimators':hp.choice('n_estimators', np.arange(400, 1200, 10, dtype=int))}

In [20]:
# def score(params):

#     model = RandomForestClassifier(**params)

#     model.fit(x_train, y_train)

#     Y_pred = model.predict(x_test)

#     score = roc_auc_score(y_test, Y_pred)
# #     score=np.sqrt(-cross_val_score(model,train_X,train_y,cv=kfolds,scoring='neg_mean_squared_error'))

#     print(score)

#     return {'loss': -score, 'status': STATUS_OK}    

# def optimize(trials, space):

#     best = fmin(score, space, algo=tpe.suggest, max_evals=100)

#     return best


# trials = Trials()

# best_params = optimize(trials, space)


# # Return the best parameters

# space_eval(space, best_params)

In [11]:
# space_eval(space,best_params)

{'bagging_temperature': 0.013585295009384768,
 'border_count': 230,
 'depth': 7,
 'iterations': 974,
 'l2_leaf_reg': 18,
 'learning_rate': 0.58,
 'random_strength': 0.65,
 'scale_pos_weight': 0.6291109899364863}

# random forest 
{'max_depth': 67,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1050}

# dry run with hypertuned classifiers

In [25]:
num_folds = 10
num_instances = len(x_train)
seed = 7
scoring = 'roc_auc'

models = []
# models.append(('CBC', CatBoostClassifier(bagging_temperature= 0.7212640190796531,border_count= 235,
#  class_weights= (0.13, 0.87),
#  depth= 7,
#  eval_metric= 'AUC',
#  iterations= 871,
#  l2_leaf_reg= 13,
#  learning_rate= 0.17,
#  random_strength= 1.57)))


# models.append(('CBC2', CatBoostClassifier(bagging_temperature= 0.7159912273814812,border_count= 247,
#  depth= 7,
#  eval_metric= 'AUC',
#  iterations= 665,
#  l2_leaf_reg= 20,
#  learning_rate= 0.67,
#  random_strength= 0.11706929793799148,
#  scale_pos_weight =0.9139659820768006)))



# models.append(('XGB1', XGBClassifier(eta= 0.2,
#                  eval_metric= 'auc',
#                  gamma= 0.9,
#                  n_estimators= 610,
#                  scale_pos_weight= 11,
#                  subsample= 0.9500000000000001)))

# models.append(('XGB2', XGBClassifier(eta= 0.5,
#                  eval_metric= 'auc',
#                  gamma= 0.9,
#                  n_estimators= 630,
#                  scale_pos_weight= 87,
#                  subsample= 1.0)))

# models.append(('XGB3', XGBClassifier(eta= 0.375,
#                  eval_metric= 'auc',
#                  gamma= 0.5,
#                  n_estimators= 900,
#                  scale_pos_weight= 28,
#                  subsample= 0.9)))

from sklearn.ensemble import AdaBoostClassifier
model=CatBoostClassifier()


kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=41)
cv_results = cross_val_score(model, x_train, y_train, cv = kfold, scoring = scoring)


cv_results

results = []
names = []
# print("Scores for each algorithm:")
# for name, model in models:
# #     resample = SMOTEENN()
# #     steps = [('r', resample), ('m', model)]
# #     pipeline = Pipeline(steps=steps)
#     kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=41)
#     cv_results = cross_val_score(model, x_test, y_test, cv = kfold, scoring = scoring)
#     results.append(cv_results)
#     names.append(name)
#     print(name, cv_results)
#     print()

Learning rate set to 0.094932
0:	learn: 0.6247441	total: 68.9ms	remaining: 1m 8s
1:	learn: 0.5703646	total: 102ms	remaining: 51s
2:	learn: 0.5294902	total: 131ms	remaining: 43.5s
3:	learn: 0.4959374	total: 172ms	remaining: 42.7s
4:	learn: 0.4677699	total: 220ms	remaining: 43.8s
5:	learn: 0.4473925	total: 274ms	remaining: 45.4s
6:	learn: 0.4307260	total: 322ms	remaining: 45.7s
7:	learn: 0.4179441	total: 372ms	remaining: 46.2s
8:	learn: 0.4076701	total: 433ms	remaining: 47.7s
9:	learn: 0.3985983	total: 487ms	remaining: 48.2s
10:	learn: 0.3918288	total: 542ms	remaining: 48.7s
11:	learn: 0.3863316	total: 597ms	remaining: 49.2s
12:	learn: 0.3813848	total: 659ms	remaining: 50s
13:	learn: 0.3775218	total: 712ms	remaining: 50.2s
14:	learn: 0.3741994	total: 773ms	remaining: 50.8s
15:	learn: 0.3716398	total: 830ms	remaining: 51s
16:	learn: 0.3696147	total: 896ms	remaining: 51.8s
17:	learn: 0.3673651	total: 965ms	remaining: 52.6s
18:	learn: 0.3643345	total: 1.01s	remaining: 52.4s
19:	learn: 0.363

165:	learn: 0.2997553	total: 7.58s	remaining: 38.1s
166:	learn: 0.2995095	total: 7.62s	remaining: 38s
167:	learn: 0.2990242	total: 7.69s	remaining: 38.1s
168:	learn: 0.2987480	total: 7.73s	remaining: 38s
169:	learn: 0.2985554	total: 7.77s	remaining: 37.9s
170:	learn: 0.2980882	total: 7.82s	remaining: 37.9s
171:	learn: 0.2979151	total: 7.86s	remaining: 37.8s
172:	learn: 0.2976796	total: 7.9s	remaining: 37.8s
173:	learn: 0.2973462	total: 7.95s	remaining: 37.7s
174:	learn: 0.2971271	total: 7.99s	remaining: 37.7s
175:	learn: 0.2967470	total: 8.04s	remaining: 37.6s
176:	learn: 0.2964542	total: 8.07s	remaining: 37.5s
177:	learn: 0.2961634	total: 8.12s	remaining: 37.5s
178:	learn: 0.2958552	total: 8.15s	remaining: 37.4s
179:	learn: 0.2955927	total: 8.19s	remaining: 37.3s
180:	learn: 0.2954145	total: 8.24s	remaining: 37.3s
181:	learn: 0.2946138	total: 8.29s	remaining: 37.3s
182:	learn: 0.2944759	total: 8.34s	remaining: 37.2s
183:	learn: 0.2942627	total: 8.39s	remaining: 37.2s
184:	learn: 0.294

326:	learn: 0.2638336	total: 14.6s	remaining: 30s
327:	learn: 0.2636155	total: 14.6s	remaining: 30s
328:	learn: 0.2634728	total: 14.7s	remaining: 29.9s
329:	learn: 0.2632304	total: 14.7s	remaining: 29.9s
330:	learn: 0.2630640	total: 14.8s	remaining: 29.8s
331:	learn: 0.2629363	total: 14.8s	remaining: 29.8s
332:	learn: 0.2627959	total: 14.8s	remaining: 29.7s
333:	learn: 0.2627220	total: 14.9s	remaining: 29.7s
334:	learn: 0.2625231	total: 14.9s	remaining: 29.6s
335:	learn: 0.2623309	total: 14.9s	remaining: 29.5s
336:	learn: 0.2622260	total: 15s	remaining: 29.5s
337:	learn: 0.2619780	total: 15s	remaining: 29.4s
338:	learn: 0.2617928	total: 15.1s	remaining: 29.4s
339:	learn: 0.2616441	total: 15.1s	remaining: 29.3s
340:	learn: 0.2615325	total: 15.2s	remaining: 29.3s
341:	learn: 0.2613853	total: 15.2s	remaining: 29.3s
342:	learn: 0.2611710	total: 15.3s	remaining: 29.2s
343:	learn: 0.2610467	total: 15.3s	remaining: 29.2s
344:	learn: 0.2608324	total: 15.3s	remaining: 29.1s
345:	learn: 0.260681

487:	learn: 0.2378606	total: 22.1s	remaining: 23.2s
488:	learn: 0.2377488	total: 22.2s	remaining: 23.2s
489:	learn: 0.2375752	total: 22.2s	remaining: 23.1s
490:	learn: 0.2373649	total: 22.2s	remaining: 23.1s
491:	learn: 0.2371744	total: 22.3s	remaining: 23s
492:	learn: 0.2370880	total: 22.3s	remaining: 23s
493:	learn: 0.2369008	total: 22.4s	remaining: 23s
494:	learn: 0.2367704	total: 22.5s	remaining: 22.9s
495:	learn: 0.2365815	total: 22.5s	remaining: 22.9s
496:	learn: 0.2364090	total: 22.6s	remaining: 22.8s
497:	learn: 0.2362603	total: 22.6s	remaining: 22.8s
498:	learn: 0.2361434	total: 22.7s	remaining: 22.8s
499:	learn: 0.2360616	total: 22.7s	remaining: 22.7s
500:	learn: 0.2359047	total: 22.8s	remaining: 22.7s
501:	learn: 0.2357695	total: 22.8s	remaining: 22.6s
502:	learn: 0.2356468	total: 22.9s	remaining: 22.6s
503:	learn: 0.2355159	total: 22.9s	remaining: 22.6s
504:	learn: 0.2353834	total: 23s	remaining: 22.5s
505:	learn: 0.2352993	total: 23s	remaining: 22.5s
506:	learn: 0.2351635	

649:	learn: 0.2173664	total: 30.9s	remaining: 16.7s
650:	learn: 0.2172200	total: 31s	remaining: 16.6s
651:	learn: 0.2170785	total: 31s	remaining: 16.6s
652:	learn: 0.2169475	total: 31.1s	remaining: 16.5s
653:	learn: 0.2168389	total: 31.1s	remaining: 16.5s
654:	learn: 0.2166996	total: 31.2s	remaining: 16.4s
655:	learn: 0.2165938	total: 31.3s	remaining: 16.4s
656:	learn: 0.2164965	total: 31.3s	remaining: 16.3s
657:	learn: 0.2163114	total: 31.4s	remaining: 16.3s
658:	learn: 0.2162164	total: 31.4s	remaining: 16.2s
659:	learn: 0.2160354	total: 31.5s	remaining: 16.2s
660:	learn: 0.2159051	total: 31.5s	remaining: 16.2s
661:	learn: 0.2158434	total: 31.6s	remaining: 16.1s
662:	learn: 0.2157253	total: 31.6s	remaining: 16.1s
663:	learn: 0.2156106	total: 31.7s	remaining: 16s
664:	learn: 0.2154697	total: 31.7s	remaining: 16s
665:	learn: 0.2153946	total: 31.8s	remaining: 15.9s
666:	learn: 0.2152256	total: 31.8s	remaining: 15.9s
667:	learn: 0.2150916	total: 31.9s	remaining: 15.8s
668:	learn: 0.214992

...

224:	learn: 0.2849711	total: 9.49s	remaining: 32.7s
225:	learn: 0.2847520	total: 9.53s	remaining: 32.6s
226:	learn: 0.2845806	total: 9.56s	remaining: 32.6s
227:	learn: 0.2843890	total: 9.6s	remaining: 32.5s
228:	learn: 0.2842564	total: 9.64s	remaining: 32.5s
229:	learn: 0.2840158	total: 9.68s	remaining: 32.4s
230:	learn: 0.2838299	total: 9.72s	remaining: 32.4s
231:	learn: 0.2835580	total: 9.76s	remaining: 32.3s
232:	learn: 0.2830684	total: 9.8s	remaining: 32.3s
233:	learn: 0.2826447	total: 9.84s	remaining: 32.2s
234:	learn: 0.2823939	total: 9.88s	remaining: 32.2s
235:	learn: 0.2821043	total: 9.91s	remaining: 32.1s
236:	learn: 0.2818437	total: 9.96s	remaining: 32.1s
237:	learn: 0.2816311	total: 10s	remaining: 32s
238:	learn: 0.2815309	total: 10.1s	remaining: 32s
239:	learn: 0.2813209	total: 10.1s	remaining: 32s
240:	learn: 0.2810085	total: 10.1s	remaining: 31.9s
241:	learn: 0.2807608	total: 10.2s	remaining: 31.9s
242:	learn: 0.2805316	total: 10.2s	remaining: 31.8s
243:	learn: 0.2802755	

383:	learn: 0.2527047	total: 17.2s	remaining: 27.5s
384:	learn: 0.2525617	total: 17.2s	remaining: 27.5s
385:	learn: 0.2524221	total: 17.3s	remaining: 27.5s
386:	learn: 0.2522658	total: 17.4s	remaining: 27.5s
387:	learn: 0.2521389	total: 17.4s	remaining: 27.5s
388:	learn: 0.2520112	total: 17.5s	remaining: 27.4s
389:	learn: 0.2518725	total: 17.5s	remaining: 27.4s
390:	learn: 0.2516965	total: 17.6s	remaining: 27.4s
391:	learn: 0.2514984	total: 17.6s	remaining: 27.3s
392:	learn: 0.2513777	total: 17.7s	remaining: 27.3s
393:	learn: 0.2512318	total: 17.7s	remaining: 27.2s
394:	learn: 0.2510891	total: 17.7s	remaining: 27.2s
395:	learn: 0.2508101	total: 17.8s	remaining: 27.1s
396:	learn: 0.2506061	total: 17.9s	remaining: 27.1s
397:	learn: 0.2504460	total: 17.9s	remaining: 27.1s
398:	learn: 0.2502452	total: 18s	remaining: 27.1s
399:	learn: 0.2500370	total: 18.1s	remaining: 27.1s
400:	learn: 0.2499164	total: 18.1s	remaining: 27s
401:	learn: 0.2497356	total: 18.2s	remaining: 27s
402:	learn: 0.2495

542:	learn: 0.2300835	total: 25.8s	remaining: 21.7s
543:	learn: 0.2299547	total: 25.9s	remaining: 21.7s
544:	learn: 0.2298577	total: 25.9s	remaining: 21.6s
545:	learn: 0.2297838	total: 26s	remaining: 21.6s
546:	learn: 0.2296655	total: 26s	remaining: 21.6s
547:	learn: 0.2295339	total: 26.1s	remaining: 21.5s
548:	learn: 0.2294630	total: 26.1s	remaining: 21.5s
549:	learn: 0.2293545	total: 26.2s	remaining: 21.4s
550:	learn: 0.2292759	total: 26.3s	remaining: 21.4s
551:	learn: 0.2291682	total: 26.3s	remaining: 21.4s
552:	learn: 0.2290056	total: 26.4s	remaining: 21.3s
553:	learn: 0.2288566	total: 26.4s	remaining: 21.3s
554:	learn: 0.2286682	total: 26.5s	remaining: 21.3s
555:	learn: 0.2285851	total: 26.6s	remaining: 21.2s
556:	learn: 0.2284917	total: 26.6s	remaining: 21.2s
557:	learn: 0.2283367	total: 26.7s	remaining: 21.1s
558:	learn: 0.2281709	total: 26.7s	remaining: 21.1s
559:	learn: 0.2280243	total: 26.8s	remaining: 21.1s
560:	learn: 0.2279489	total: 26.9s	remaining: 21s
561:	learn: 0.2278

702:	learn: 0.2101897	total: 33.3s	remaining: 14.1s
703:	learn: 0.2100753	total: 33.4s	remaining: 14s
704:	learn: 0.2099756	total: 33.4s	remaining: 14s
705:	learn: 0.2098822	total: 33.4s	remaining: 13.9s
706:	learn: 0.2096772	total: 33.5s	remaining: 13.9s
707:	learn: 0.2094390	total: 33.5s	remaining: 13.8s
708:	learn: 0.2093835	total: 33.5s	remaining: 13.8s
709:	learn: 0.2092787	total: 33.6s	remaining: 13.7s
710:	learn: 0.2092249	total: 33.6s	remaining: 13.7s
711:	learn: 0.2091697	total: 33.7s	remaining: 13.6s
712:	learn: 0.2091010	total: 33.7s	remaining: 13.6s
713:	learn: 0.2090372	total: 33.7s	remaining: 13.5s
714:	learn: 0.2089171	total: 33.8s	remaining: 13.5s
715:	learn: 0.2088314	total: 33.8s	remaining: 13.4s
716:	learn: 0.2087296	total: 33.9s	remaining: 13.4s
717:	learn: 0.2086276	total: 33.9s	remaining: 13.3s
718:	learn: 0.2085388	total: 33.9s	remaining: 13.3s
719:	learn: 0.2084711	total: 34s	remaining: 13.2s
720:	learn: 0.2084058	total: 34s	remaining: 13.2s
721:	learn: 0.208160

863:	learn: 0.1945915	total: 40.3s	remaining: 6.34s
864:	learn: 0.1944495	total: 40.3s	remaining: 6.29s
865:	learn: 0.1943527	total: 40.4s	remaining: 6.25s
866:	learn: 0.1942759	total: 40.4s	remaining: 6.2s
867:	learn: 0.1941854	total: 40.4s	remaining: 6.15s
868:	learn: 0.1941290	total: 40.5s	remaining: 6.1s
869:	learn: 0.1940223	total: 40.5s	remaining: 6.05s
870:	learn: 0.1939013	total: 40.6s	remaining: 6.01s
871:	learn: 0.1938044	total: 40.6s	remaining: 5.96s
872:	learn: 0.1937736	total: 40.6s	remaining: 5.91s
873:	learn: 0.1936836	total: 40.7s	remaining: 5.87s
874:	learn: 0.1936079	total: 40.7s	remaining: 5.82s
875:	learn: 0.1935574	total: 40.8s	remaining: 5.77s
876:	learn: 0.1934683	total: 40.8s	remaining: 5.72s
877:	learn: 0.1933809	total: 40.9s	remaining: 5.68s
878:	learn: 0.1933013	total: 40.9s	remaining: 5.63s
879:	learn: 0.1931895	total: 40.9s	remaining: 5.58s
880:	learn: 0.1930847	total: 41s	remaining: 5.54s
881:	learn: 0.1930020	total: 41.1s	remaining: 5.49s
882:	learn: 0.19

In [24]:
cv_results

array([0.67053673, 0.66328346, 0.67000877, 0.67533051, 0.66766531,
       0.6643579 , 0.65930991, 0.6573103 , 0.67171937, 0.66818869])

# stacking

In [7]:
# rf=RandomForestClassifier(max_depth=67,min_samples_leaf=1,min_samples_split=2,n_estimators=1050)
# xgb=XGBClassifier(gamma= 0.77,eta=0.39, n_estimators= 880,subsample=0.87)
# cat=CatBoostClassifier(bagging_temperature= 0.013585295009384768,border_count= 230,depth= 7,iterations= 974,
#              l2_leaf_reg= 18,
#              learning_rate= 0.58,
#              random_strength= 0.65,
#              scale_pos_weight= 0.6291109899364863)
# lgbm=LGBMClassifier(bagging_fraction= 0.6362408347656274,
#              learning_rate= 0.41000000000000003,
#              max_depth= 12,
#              min_child_weight= 1.9928502540411177,
#              n_estimators= 910,
#              subsample= 0.56)

# a=LogisticRegression()
# b=LinearDiscriminantAnalysis()
# c=KNeighborsClassifier()
# d=DecisionTreeClassifier()
# e=MLPClassifier(solver='lbfgs')
# f=GaussianNB()
# g=GradientBoostingClassifier()
# h=XGBClassifier()
# i=LGBMClassifier()


cat=CatBoostClassifier(bagging_temperature= 0.7212640190796531,border_count= 235,
 class_weights= (0.13, 0.87),
 depth= 7,
 eval_metric= 'AUC',
 iterations= 871,
 l2_leaf_reg= 13,
 learning_rate= 0.17,
 random_strength= 1.57)
                 

    
    
    
xgb=XGBClassifier(eta= 0.375,
 eval_metric= 'auc',
 gamma= 0.5,
 n_estimators= 900,
 scale_pos_weight= 28,
 subsample= 0.9)

In [31]:
stack_gen = StackingCVClassifier(classifiers=( cat, xgb),
                                meta_classifier=xgb,
                                use_features_in_secondary=True)

In [8]:


# stack_gen_model = stack_gen.fit(np.array(xtrain), np.array(ytrain))
xgb_model=xgb.fit(xtrain, ytrain)
# # rf_model=rf.fit(x_train, y_train)
# lgbm_model=lgbm.fit(x_train, y_train)
# cat_model=cat.fit(x_train, y_train)

/home/jaidev/anaconda3/envs/my_env/lib/python3.9/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [9]:
test_data=test_data.rename(columns={'id':'Id'})

# results


In [11]:

result = pd.DataFrame()

# make a prediction on a new row of data

result['id'] = test_data['Id']

result['risk_flag'] = pd.DataFrame(xgb_model.predict(test_data))

In [12]:
result.to_csv('predictions_xgb.csv',index=False)